# LGBM - dora/Training on a rolling dataset

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import sys
import xgboost as xgb
import lightgbm as lgb
from datetime import datetime

NUMBER_OF_LAGS = 4

sys.path.append("../main/datasets/")
!ls  ../main/datasets/

1.0v  1.0v.zip


In [2]:
sys.path.append("../dora/models")
from utils import *

<hr>

## Defining metrics

Baseline_score function

In [3]:
def baseline_score(prediction, target, simulatedPrice):
    prediction = prediction.astype(int)

    return np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice)

Evaluation Metric

In [4]:
def feval(prediction, dtrain):
    
    prediction = prediction.astype(int)
    target = dtrain.get_label()

    simulatedPrice = dtrain.get_weight()
    
    return 'feval', np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice), True

<hr>

## Building our dataset
This notebook makes this step cleaner than the previous versions. So It'll be tidier and shorter than before!

In [5]:
infos, items, orders = read_data("../main/datasets/")
print("Sanity checks...", infos.shape, items.shape, orders.shape)

Sanity checks... (10463, 3) (10463, 8) (2181955, 5)


In [6]:
# Changing our time signatures
process_time(orders)

In [7]:
df = dataset_builder(orders, items)

<hr>

## Feature building

**Adding 'is_new'**

In [8]:
# This cell adds a feature responsible for indicating if in the current week
# a given item has its first appearance.
orders_sorted_by_week = orders.sort_values('group_backwards', ascending=False)
weeks_grouped_by_items = orders_sorted_by_week.groupby('itemID', as_index=False)
items_first_appearance = weeks_grouped_by_items.first()[['itemID', 'group_backwards']]

items_first_appearance.rename(columns={'group_backwards':'first_appearance'}, inplace=True)

df['is_new'] = 0

df = pd.merge(df, items_first_appearance, left_on=['itemID'], right_on=['itemID'], how='left', validate='m:1')

df.loc[df['first_appearance'] == df['group_backwards'], 'is_new'] = 1
df.drop(columns=['first_appearance'], inplace=True)

**Cumulative sale by category**

In [9]:
# percentage_accum_cat_3 feature...
df = cumulative_sale_by_category(df, category='category3')

**Time Encoding**

In [10]:
# Encoding our weeks as a series of sines and cosines...
# This function will consider our period as a semester in a year,
# so we can try other types of time encoding later!
df = time_encoder(df, 'group_backwards', 26)

# new feature: mean price of older pair_week

In [11]:
#getting mean price
meanPrice = orders.groupby(['itemID','group_backwards'],as_index=False).salesPrice.mean()
meanPrice.columns = ['itemID','group_backwards','meanPrice_1']

#shifting one week
df = pd.merge(df,meanPrice,on=['itemID','group_backwards'],how='left')
df['meanPrice_1'] = df.groupby('itemID')['meanPrice_1'].shift(1)

#shifting more weeks and calculating diff
for i in range(1, NUMBER_OF_LAGS ):
    df[f'meanPrice_{i+1}'] = df.groupby('itemID')['meanPrice_1'].shift(i)

    df[f'meanPrice_diff_{i+1}'] = df.groupby('itemID')[f'meanPrice_{i+1}'].diff()

In [12]:
#modelos literalmente nao mudou, talvez tentar no search grid
df['diff_retail_mean_price'] = df['recommendedRetailPrice'] - df['meanPrice_1']

#essa n testei
df['percent_retail_mean_price'] = df['meanPrice_1'] / df['recommendedRetailPrice'] * 100

**Lags and diffs**

In [13]:
# This cell lags and diffs our feature 'orderSum'
shifting = df.copy()

for i in range(1, NUMBER_OF_LAGS + 1):
    # Carrying the data of weeks t-1
    shifting[f'orderSum_{i}'] = shifting.groupby('itemID')['orderSum'].shift(i)

    
    # Getting the difference of the orders and promotions between weeks t-1 and t-2...
    shifting[f'orderSum_diff_{i}'] = shifting.groupby('itemID')[f'orderSum_{i}'].diff()
    

**Rolling window "orderSum"**

In [14]:
%%time
# This cell creates rolling-window features based on 'orderSum' in our dataset!
item_group = shifting.groupby(["itemID", "group_backwards"]).agg({'orderSum':'sum'})

# We'll .shift(-1) because it sorts our "group_backwards", 
# so doing .shift(1) would cause a HUGE dataleak.
aux_shifting = item_group.groupby('itemID')[['orderSum']].shift(-1)

aux_shifting.sort_values(['itemID', 'group_backwards'], ascending=[True, False], inplace=True)

for i in range(3):
    rolled_window = aux_shifting.groupby(['itemID'], as_index=False)[['orderSum']].rolling(2 ** i).mean()
    rolled_window.rename(columns={'orderSum':f"orderSum_mean_rolled_{i}"}, inplace=True)
    shifting = pd.merge(shifting, rolled_window, left_on=['itemID', 'group_backwards'], right_on=['itemID', 'group_backwards'])

CPU times: user 46.2 s, sys: 314 ms, total: 46.5 s
Wall time: 46.4 s


In [15]:
# LGBM Says on docs that it automatically handles zero values as NaN,
# so we'll keep this standard...
shifting.fillna(0, inplace=True)

<hr>

## Maximum error
The maximum error we could get in this dataset would be just guessing the mean of our sales from weeks 1 to 12, and that's what the cell below is computing.

In [16]:
worst_possible_prediction = shifting.loc[shifting.group_backwards > 1]['orderSum'].mean()
prediction = np.full(shifting.loc[shifting.group_backwards == 1]['orderSum'].shape, worst_possible_prediction) # Array filled with the mean...
target = shifting.loc[shifting.group_backwards == 1]['orderSum']
print("Guessing the mean of 'orderSum' for all items in target", mse(target, prediction) ** 0.5)

Guessing the mean of 'orderSum' for all items in target 118.20105838913783


<hr>

## Dataset Splitting (Train until week 3 / Val. week 2/ Test week 1)
All my experiments will use weeks 13 to 3 as a train set, week 2 as our validation set and week 1 as a test set.

In [17]:
results = []
for i in range(5, 1, -1):
    print(f"TRAIN UNTIL : {i} VAL : {i - 1}")
    train = shifting.loc[shifting.group_backwards >= i]
    val = shifting.loc[shifting.group_backwards == i - 1]
    # test = shifting.loc[shifting.group_backwards == 1]

    weights = infos.set_index('itemID')['simulationPrice'].to_dict()

    w_train = train['itemID'].map(weights)
    w_val = val['itemID'].map(weights)

    # I recommend to the other members of the team keeping the
    # datatypes of our datasets as Pandas DataFrames instead of Numpy,
    # since It will easier to use Boosting Analysis frameworks
    y_train = train['orderSum']
    y_val = val['orderSum']
    X_train = train.drop(columns=["orderSum"])
    X_val = val.drop(columns=["orderSum"])

    params = {
    #           "objective" : "poisson",
              "objective" : "l1",
              "metric" :"rmse",
              "learning_rate" : 0.5,
              'verbosity': 1,
              'max_depth': 6,
              'num_leaves': 32,
              "min_data_in_leaf":3000,
             }

    lgbtrain = lgb.Dataset(X_train, label = y_train, weight=w_train, categorical_feature=[2, 3, 5, 6, 7, 9])
    lgbvalid = lgb.Dataset(X_val, label = y_val, weight=w_val, categorical_feature=[2, 3, 5, 6, 7, 9])

    num_round = 1000
    model = lgb.train(params,
                      lgbtrain,
                      num_round,
                      valid_sets = [lgbtrain, lgbvalid], 
                      verbose_eval=5,
                      early_stopping_rounds=5,
    #                   fobj=objective,
                      feval=feval,
                     )
    results.append(model.best_score)


TRAIN UNTIL : 5 VAL : 4


/home/g/.local/lib/python3.6/site-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 5 rounds
[5]	training's rmse: 38.9961	training's feval: 4.07323e+06	valid_1's rmse: 38.6086	valid_1's feval: 451570
[10]	training's rmse: 38.7978	training's feval: 4.69228e+06	valid_1's rmse: 38.331	valid_1's feval: 499882
[15]	training's rmse: 38.6515	training's feval: 5.09358e+06	valid_1's rmse: 38.138	valid_1's feval: 543597
[20]	training's rmse: 40.565	training's feval: -4.8161e+06	valid_1's rmse: 40.3252	valid_1's feval: -530174
Early stopping, best iteration is:
[15]	training's rmse: 38.6515	training's feval: 5.09358e+06	valid_1's rmse: 38.138	valid_1's feval: 543597
TRAIN UNTIL : 4 VAL : 3
Training until validation scores don't improve for 5 rounds
[5]	training's rmse: 38.8517	training's feval: 4.8551e+06	valid_1's rmse: 44.1976	valid_1's feval: 522840
[10]	training's rmse: 38.4937	training's feval: 5.95412e+06	valid_1's rmse: 43.7568	valid_1's feval: 683262
[15]	training's rmse: 38.4907	training's feval: 5.99017e+06	valid_1's r

In [18]:
for i in results:
    print(i['valid_1']['feval'])
results

543596.5905374885
689285.5871709584
721535.2644547463
805720.735173726


[defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('rmse', 38.65153747903404),
                           ('feval', 5093575.850479722)]),
              'valid_1': OrderedDict([('rmse', 38.13800449535862),
                           ('feval', 543596.5905374885)])}),
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('rmse', 38.49078122640976),
                           ('feval', 5990167.6235336475)]),
              'valid_1': OrderedDict([('rmse', 43.753529918227805),
                           ('feval', 689285.5871709584)])}),
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('rmse', 38.250455852842904),
                           ('feval', 8922722.037940454)]),
              'valid_1': OrderedDict([('rmse', 43.298291976505006),
                           ('feval', 721535.2644547463)])}),
 defaultdict(collections.OrderedDict,
             {'training': OrderedDict([('rmse', 38.199840556371164),
          

<hr>

## Dataset Splitting (Retraining until week 2 and test with week 1)
All my experiments will use weeks 13 to 2 as a train set and week 1 as test

In [19]:
train = shifting.loc[shifting.group_backwards >= 2]
test = shifting.loc[shifting.group_backwards == 1]

weights = infos.set_index('itemID')['simulationPrice'].to_dict()

w_train = train['itemID'].map(weights)

In [20]:
# I recommend to the other members of the team keeping the
# datatypes of our datasets as Pandas DataFrames instead of Numpy,
# since It will easier to use Boosting Analysis frameworks
y_train = train['orderSum']
X_train = train.drop(columns=["orderSum"])

In [21]:
params = {
    #           "objective" : "poisson",
              "objective" : "l1",
              "metric" :"rmse",
              "learning_rate" : 0.5,
              'verbosity': 1,
              'max_depth': 6,
              'num_leaves': 32,
              "min_data_in_leaf":3000,
             }

lgbtrain = lgb.Dataset(X_train, label = y_train, weight=w_train, categorical_feature=[2, 3, 5, 6, 7, 9])

model = lgb.train(params,
                  lgbtrain,
                  model.best_iteration,
                  valid_sets = [lgbtrain], 
                  valid_names = ['train'],
                  verbose_eval=5,
                  early_stopping_rounds=5,
#                   fobj=objective,
                  feval=feval,
                  
                 )

Training until validation scores don't improve for 5 rounds
[5]	train's rmse: 39.6125	train's feval: 6.41924e+06
[10]	train's rmse: 38.1998	train's feval: 1.03691e+07
Did not meet early stopping. Best iteration is:
[10]	train's rmse: 38.1998	train's feval: 1.03691e+07


<hr>

### Utilities

**Predicting at test time**

In [22]:
y_test = test['orderSum']
X_test = test.drop(columns=["orderSum"])
final_predictions = model.predict(X_test)

In [23]:
final_predictions

array([0.75      , 0.5       , 1.96185332, ..., 0.        , 0.        ,
       0.        ])

In [24]:
final_predictions[final_predictions < 0] = 0

In [25]:
final_predictions

array([0.75      , 0.5       , 1.96185332, ..., 0.        , 0.        ,
       0.        ])

In [26]:
baseline_score(final_predictions, y_test.values, infos['simulationPrice'])

805720.7339999998

# Post Processing (based on Bruno's 1.8 Notebook)

In [27]:
# Weekpair is negative, so this works:
first_fortnight_item = orders.sort_values("group_backwards",
                                     ascending=False)\
                          .groupby(["itemID"])["group_backwards"].first()

In [28]:
new_items_value = 10  # The mode seems too high, so go with 10...
idx = X_test["itemID"].isin(first_fortnight_item[first_fortnight_item == 1].index)
final_predictions[idx] = new_items_value

**Baseline calculation**

In [29]:
baseline_score(final_predictions, y_test.values, infos['simulationPrice'])

457681.0